You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Filter Operation:
o	Perform a "filter" operation on the retrieved launch data based on specific criteria.
o	Filter Logic: Filter the list of launches based on launch year and launch success status. Only include launches that match the specified criteria.
o	The result should be a list of dictionaries, containing only the launch records that satisfy the filter conditions.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URL and the filtering criteria, making them easily modifiable. Use parameters like: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
	FILTER_YEAR = 2019 (Set to an integer year, or None to disable year filtering)
	FILTER_SUCCESS = True (Set to True, False, or None to disable success filtering)
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, filtering criteria being applied, number of launches before filtering, number of launches after filtering, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + filtering). Print this time after the filtering operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting filtered list from the filter operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting filtered JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
# Databricks Notebook
# COMMAND ----------

import requests
import json
import logging
import time

# COMMAND ----------

# Control Parameters
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
FILTER_YEAR = 2019  # Set to an integer year, or None to disable year filtering
FILTER_SUCCESS = True  # Set to True, False, or None to disable success filtering
HTTPBIN_UPLOAD_URL = "https://httpbin.org/post"

# Logging Configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Start measuring total execution time
start_time_total = time.time()

# COMMAND ----------

# 1. Data Retrieval from SpaceX API
logging.info(f"Starting data retrieval from: {API_ENDPOINT_URL}")
start_time_retrieval = time.time()
all_launches_data = []

try:
    response = requests.get(API_ENDPOINT_URL, timeout=10)
    response.raise_for_status()  # Raise an exception for non-200 status codes
    all_launches_data = response.json()
    logging.info(f"Successfully retrieved {len(all_launches_data)} launches.")
except requests.exceptions.Timeout as e:
    logging.error(f"Timeout error during API call: {e}")
except requests.exceptions.RequestException as e:
    logging.error(f"Error during API call: {e}")
except json.JSONDecodeError as e:
    logging.error(f"Error decoding JSON response: {e}")
finally:
    end_time_retrieval = time.time()
    retrieval_time = end_time_retrieval - start_time_retrieval
    logging.info(f"Data retrieval completed in {retrieval_time:.2f} seconds.")

# COMMAND ----------

# 2. Filter Operation
logging.info("Starting filter operation.")
logging.info(f"Filtering criteria - Year: {FILTER_YEAR}, Success: {FILTER_SUCCESS}")

start_time_filtering = time.time()
filtered_launches = []
num_launches_before_filtering = len(all_launches_data)
logging.info(f"Number of launches before filtering: {num_launches_before_filtering}")

if all_launches_data:
    for launch in all_launches_data:
        year_condition = True
        success_condition = True

        if FILTER_YEAR is not None:
            try:
                launch_year = int(launch.get('launch_year'))
                year_condition = launch_year == FILTER_YEAR
            except (ValueError, TypeError):
                year_condition = False
                logging.warning(f"Could not parse launch year for launch with flight_number: {launch.get('flight_number')}")

        if FILTER_SUCCESS is not None:
            success_condition = launch.get('launch_success') == FILTER_SUCCESS

        if year_condition and success_condition:
            filtered_launches.append(launch)

end_time_filtering = time.time()
filtering_time = end_time_filtering - start_time_filtering
logging.info(f"Number of launches after filtering: {len(filtered_launches)}")
logging.info(f"Filter operation completed in {filtering_time:.2f} seconds.")
print(f"Time taken for data retrieval and filtering: {retrieval_time + filtering_time:.2f} seconds")

# COMMAND ----------

# 5. Upload Result
logging.info(f"Starting upload to: {HTTPBIN_UPLOAD_URL}")
start_time_upload = time.time()
upload_successful = False

if filtered_launches:
    try:
        json_payload = json.dumps(filtered_launches)
        headers = {'Content-Type': 'application/json'}
        response = requests.post(HTTPBIN_UPLOAD_URL, headers=headers, data=json_payload, timeout=10)
        response.raise_for_status()
        upload_successful = True
        logging.info(f"Upload to {HTTPBIN_UPLOAD_URL} successful. Status code: {response.status_code}")
        logging.debug(f"Response from {HTTPBIN_UPLOAD_URL}: {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during upload to {HTTPBIN_UPLOAD_URL}: {e}")
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding response from {HTTPBIN_UPLOAD_URL}: {e}")
else:
    logging.info("No launches to upload after filtering.")

end_time_upload = time.time()
upload_time = end_time_upload - start_time_upload
logging.info(f"Upload operation completed in {upload_time:.2f} seconds.")
logging.info(f"Upload outcome: {'Success' if upload_successful else 'Failure'}")

# COMMAND ----------

# Measure total execution time
end_time_total = time.time()
total_execution_time = end_time_total - start_time_total
print(f"Total script execution time: {total_execution_time:.2f} seconds")

# COMMAND ----------